In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 5
height = 4
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 30

puzzle.naming = {
    None: ' ',
    'e': '_',
}

puzzle.domain_program += """
    value(e).
    num(2..6).
    value(N) :- num(N).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
    :- puzzle(C,e), cell(C).
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['cell_distance']
puzzle.solution_program += enc_library['adjacent_cells']
puzzle.solution_program += """
    1 { arrow(C1,C2) : cell_distance(C1,C2,D), solution(C2,E), num(E) } 1 :- solution(C1,D), num(D).
    :- solution(C1,E), num(E), not 1 { arrow(C2,C1) : solution(C2,D), num(D) } 1.
    
    :- arrow(C1,C2), arrow(C2,C1).
    
    :- adjacent_cells(C1,C2), solution(C1,D), num(D), solution(C2,E), num(E).
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
.....
Stopped after solving timeout..
Solving time: 32.51 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

| | |5| | |
|3| | | |2|
| | | | | |
| | | | |5|


In [5]:
print(puzzle.pretty_repr_solution())

|_|_|5|_|_|
|3|_|_|_|2|
|_|_|3|_|_|
|2|_|_|_|5|
